In [53]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [139]:
df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/defaultCredit/data.csv')
target_clm = 'default.payment.next.month'
#get numerical columns
dtyp_df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/defaultCredit/fold0/data_types.csv')
num_types = dtyp_df[dtyp_df.type.isin(['pos','count', 'real'])].index
num_types = df.columns[num_types]
cat_types = dtyp_df[~dtyp_df.type.isin(['pos','count', 'real'])].index
num_categories = dtyp_df[~dtyp_df.type.isin(['pos','count', 'real'])].dim
cat_types = df.columns[cat_types]
# cat_types = cat_types.drop(target_clm)

In [142]:
chk = pd.DataFrame(columns=cat_types)
chk

,SEX,EDUCATION,MARRIAGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default.payment.next.month


array([ 3,  7,  4, 11, 11, 11, 11, 11, 11,  2])

In [163]:
chk.loc[0] = num_categories.values

In [164]:
chk

,SEX,EDUCATION,MARRIAGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default.payment.next.month
0,3,7,4,11,11,11,11,11,11,2


In [131]:
cat_types = cat_types.to_series()
cat_types

SEX                SEX
EDUCATION    EDUCATION
MARRIAGE      MARRIAGE
PAY_0            PAY_0
PAY_2            PAY_2
PAY_3            PAY_3
PAY_4            PAY_4
PAY_5            PAY_5
PAY_6            PAY_6
dtype: object

In [128]:
type(num_categories)

pandas.core.series.Series

In [117]:
scaler = StandardScaler()
# first we need to detect numerical clms and then scale, this is incorrect
df[num_types] = scaler.fit_transform(df[num_types])
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,-1.136720,2,2,1,-1.246020,2,2,-1,-1,-2,...,-0.672497,-0.663059,-0.652724,-0.341942,-0.227086,-0.296801,-0.308063,-0.314136,-0.293382,1
1,-0.365981,2,2,2,-1.029047,-1,2,0,0,0,...,-0.621636,-0.606229,-0.597966,-0.341942,-0.213588,-0.240005,-0.244230,-0.314136,-0.180878,1
2,-0.597202,2,2,2,-0.161156,0,0,0,0,0,...,-0.449730,-0.417188,-0.391630,-0.250292,-0.191887,-0.240005,-0.244230,-0.248683,-0.012122,0
3,-0.905498,2,2,1,0.164303,0,0,0,0,0,...,-0.232373,-0.186729,-0.156579,-0.221191,-0.169361,-0.228645,-0.237846,-0.244166,-0.237130,0
4,-0.905498,1,2,1,2.334029,-1,0,-1,0,0,...,-0.346997,-0.348137,-0.331482,-0.221191,1.335034,0.271165,0.266434,-0.269039,-0.255187,0


In [112]:
cat_types = cat_types.to_frame(index=False).T
# num_types = num_types.to_frame(index=False).T
# num_types.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/defaultCredit/numerical_clms.csv', \
#      index=False, header=False)
# cat_types.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/defaultCredit/categorical_clms.csv', \
#      index=False, header=False)

In [103]:
cat_types[len(cat_types.index)] = num_categories
# num_categories = num_categories.to_frame(index=False).T

In [114]:
num_categories = num_categories.to_frame().T

In [115]:
num_categories

,1,2,3,5,6,7,8,9,10,23
dim,3,7,4,11,11,11,11,11,11,2


In [52]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=142)
df_train, df_valid = train_test_split(df_train, test_size=0.1, random_state=142)
df_train.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/defaultCredit/fold0/train/data.csv', index=False)
df_valid.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/defaultCredit/fold0/valid/data.csv', index=False)
df_test.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/defaultCredit/fold0/test/data.csv', index=False)